A downside to simulation is that we are finding an approximate p-value.
How do we know how accurate the value is?

If you run the code below several times, you will notice that the p-value changes slightly each time.
How can we be sure that it won't drop below 0.05 and give us a spurious result?

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


n_trials = 10000
n_rolls_per_trial = 24

all_sixes = []
for i in range(n_trials):
    rolls = np.random.randint(1, 7, size=n_rolls_per_trial)
    num_sixes = np.sum(rolls == 6)
    all_sixes.append(num_sixes)

bins = np.arange(0, max(all_sixes) + 1.5) - 0.5
plt.hist(all_sixes, bins, color='c', edgecolor='k', weights=np.ones_like(all_sixes)/len(all_sixes))
plt.gca().set_xticks(bins + 0.5);
plt.axvline(7, color='k', linestyle='dashed', linewidth=1)
plt.xlabel('# sixes');
plt.ylabel('probability');

print('p = ', np.sum(np.array(all_sixes) >= 7)/n_trials)

This cell lets us play around with `n_trials` and see how it changes our results.
You can see that it's just a double for-loop.
The code finds the average number of 6s in 24 rolls by performing 24 x `n_trials` rolls.

We are finding the average of number of 6s through simulation many times to see how much variability there is in the result.
Just like a "trial" is rolling a die 24 times,
an "experiment" is "rolling a die 24 times `n_trials` times".
(These are not standard names.  I just made them up because I needed variable names.)

Change the value of `n_trials` to a few very different numbers and see what it does to the resulting histogram.
Do larger values of `n_trials` make the histogram narrower, or wider?

Be careful -- this code can take a while to run if you set the numbers too large.

In [ ]:
n_rolls_per_trial = 24
n_trials = 500
n_experiments = 1000  # a single 'experiment' is running n_trials times

avg_sixes = []
for i in range(n_experiments):
    total_sixes = 0
    for j in range(n_trials):
        rolls = np.random.randint(1, 7, size=n_rolls_per_trial)
        num_sixes = np.sum(rolls == 6)
        total_sixes += num_sixes
    avg_sixes.append(total_sixes/n_trials)

plt.hist(avg_sixes, color='c', edgecolor='k', weights=np.ones_like(avg_sixes)/len(avg_sixes))
plt.xlabel('# sixes');
plt.ylabel('probability');

Same idea, but now with p-values.
What kind of range of p-values will we see for different numbers of `n_trials`?
How big does `n_trials` have to be before we are almost certain not to dip below 0.05 and get a bad result?

In [ ]:
n_rolls_per_trial = 24
n_trials = 500
n_experiments = 1000  # a single 'experiment' is running n_trials times

p_values = []
for i in range(n_experiments):
    count = 0
    for j in range(n_trials):
        rolls = np.random.randint(1, 7, size=n_rolls_per_trial)
        num_sixes = np.sum(rolls == 6)
        if num_sixes >= 7:
            count += 1
    p_values.append(count/n_trials)

plt.hist(p_values, color='c', edgecolor='k', weights=np.ones_like(p_values)/len(p_values))
plt.xlabel('p-value');
plt.ylabel('probability');

We can see that large numbers of trials give us more accurate results, but what is the overall pattern?
This code plots an estimated p-value vs `n_trials`.
You can see that the estimated p-values get tighter and tighter around the true value as `n_trials` increases,
but that we see less improvement between, say, 600 and 100 than we do between 0 and 400.
Ensuring an accurate approximate p-value can require quite a few iterations.

In [ ]:
n_rolls_per_trial = 24
trial_options = range(10, 1000, 1)

ps = []
for n_trials in trial_options:
    count = 0
    for i in range(n_trials):
        rolls = np.random.randint(1, 7, size=n_rolls_per_trial)
        num_sixes = np.sum(rolls == 6)
        if num_sixes >= 7:
            count += 1
    ps.append(count/n_trials)

plt.plot(trial_options, ps, 'o')
plt.xlabel('# trials');
plt.ylabel('estimated p-value');

Can we find a numerical relationship between `n_trials` and the tightness of our approximation?
The next snippet groups our approximate p-values into bins of 50 and computes their standard deviation.

In [ ]:
std_devs = []
bins = np.arange(10, 1000, 50)

for i in range(len(bins)):
    subset = ps[i*50:(i+1)*50]
    std_devs.append(np.std(subset))

plt.plot(bins, std_devs, 'o--');
plt.xlabel('n_trials');
plt.ylabel('standard deviation');

The above plot might not look like a clear to the human eye.
Let's plot it in log-log instead.

If you plot a relationship like
$$y = ax^b$$
in log-log, you see
$$\log(y) = \log(a) + b \log(x).$$

So, if you have a straight line in a log-log plot, there is a relationship like
$$y= ax^b,$$
and the slope of the line gives you $b$.

In [ ]:
plt.loglog(bins, std_devs, 'o--', basex=2, basey=2);

The above plot is roughly a straight line with a slope of $-\frac 1 2$.
So, the standard deviation of your approximate p-values decreases as $\frac{1}{\sqrt{n\_trials}}$.
This means that if you want to decrease your standard deviation by a factor of 2,
you will need to use $2^2 = 4$ times as many trials.

This relationship is not a coincidence, and it is not unique to this example.
What you are seeing is the **Central Limit Theorem**!

In [ ]:
'''
This is just to show what a log-log plot looks like if you use exactly 1/sqrt(n) (= n^(-1/2)) as your function.
'''
# Generic log-log plot
ns = np.arange(1, 10000)
plt.loglog(ns, 1/np.sqrt(ns))